### Chatbot for game recommandation

In [ ]:
!pip install langchain
!pip install langchain_openai
!pip install --upgrade typing_extensions
!pip uninstall openai langchain langchain_openai -y
!pip install openai langchain langchain_openai --upgrade
!pip show typing_extensions


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
!pip install --upgrade typing_extensions

In [2]:
!pip uninstall openai langchain langchain_openai -y
!pip install openai langchain langchain_openai --upgrade

Found existing installation: openai 1.68.2
Uninstalling openai-1.68.2:
  Successfully uninstalled openai-1.68.2
Found existing installation: langchain 0.3.21
Uninstalling langchain-0.3.21:
  Successfully uninstalled langchain-0.3.21
Found existing installation: langchain-openai 0.3.10
Uninstalling langchain-openai-0.3.10:
  Successfully uninstalled langchain-openai-0.3.10
  Using cached openai-1.68.2-py3-none-any.whl.metadata (25 kB)
  Using cached langchain-0.3.21-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_openai-0.3.10-py3-none-any.whl.metadata (2.3 kB)
Using cached openai-1.68.2-py3-none-any.whl (606 kB)
Using cached langchain-0.3.21-py3-none-any.whl (1.0 MB)
Using cached langchain_openai-0.3.10-py3-none-any.whl (61 kB)


In [ ]:
import requests
import time
import json
import numpy as np
from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from sentence_transformers import SentenceTransformer


In [ ]:


embedding_model = SentenceTransformer('all-MiniLM-L6-v2', cache_folder="./cache")


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
import numpy as np
from scipy.spatial.distance import cosine

# Charger les jeux IGDB
with open("igdb_games_embedding.json", "r") as f:
    games = json.load(f)

def extract_game_reference(user_query):
    # on parcours tous les jeux et on vérifie si les infos sont contenu dans la description
    for game in games:
        game_name = game.get("name", "").lower()
        if game_name and game_name in user_query.lower():
            return game
    return None


# --- Fonction de recommandation basée sur la similarité et extraction d'information ---
def recommend_game(user_query):
    # On tente d'extraire une référence à un jeu depuis la requête
    ref_game = extract_game_reference(user_query)
    if ref_game and "genres" in ref_game and ref_game["genres"]:
        # Récupération des genres du jeu de référence (en minuscules)
        ref_genres = [genre["name"].lower() for genre in ref_game["genres"] if "name" in genre]
        # Filtrer les jeux pour ne retenir que ceux partageant au moins un genre
        filtered_games = []
        for game in games:
            if "genres" in game and game["genres"]:
                game_genres = [g["name"].lower() for g in game["genres"] if "name" in g]
                if any(genre in game_genres for genre in ref_genres):
                    filtered_games.append(game)
        if filtered_games:
            # Calcul de la similarité sur la sous-liste filtrée
            query_embedding = embedding_model.encode(user_query)
            similarities = [
                (game["name"], 1 - cosine(query_embedding, game["embedding"]))
                for game in filtered_games if "embedding" in game
            ]
            similarities.sort(key=lambda x: x[1], reverse=True)
            return similarities[:3]
    
    # Si aucun jeu de référence n'est détecté ou pas de genres communs, on utilise la recherche globale
    query_embedding = embedding_model.encode(user_query)
    similarities = [
        (game["name"], 1 - cosine(query_embedding, game["embedding"]))
        for game in games if "embedding" in game
    ]
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:3]

# Test
user_input = "I'm looking for an RPG with a good storyline and deep characters."
print("Jeux recommandés :", recommend_game(user_input))


Jeux recommandés : [("White : The Hero's Return", 0.689318645940777), ('The Meridian Shard', 0.6876574323494437), ('Rival Threads: Last Class Heroes', 0.6784999876762687)]


In [7]:
#!pip install langchain langchain_openai "pydantic<2" openai
#!pip install --upgrade typing_extensions

In [ ]:
class ChatSession:
    def __init__(self, memsize, llm):
        self.llm = llm
        self.memsize = memsize
        self.history = []
        self.llmID = "ASSISTANT:"
        self.userID = "USER:"
                
    def summarizeStrategy(self):
        summaryPrompt = "Résume la conversation suivante en gardant un maximum d'informations fournies par " + self.userID + " :\n"
        lastMessage, lastMessageLength = self.history[-1]
        history = ""
        for m, l in self.history[:-1]:
            history += m + "\n"
        summary = self.llm.invoke([HumanMessage(content=summaryPrompt + "\n" + history)]).content
        summaryLength = self.estimateLength(summary)
        if summaryLength + lastMessageLength > self.memsize:
            self.fifoStrategy()
            return
        self.history = [(summary, summaryLength), (lastMessage, lastMessageLength)]
    
    def fifoStrategy(self):
        while self.historyLength() > self.memsize:
            self.history = self.history[1:]
    
    def historyLength(self):
        return sum(length for _, length in self.history)
    
    def estimateLength(self, text):
        return self.llm.get_num_tokens(text)
    
    def addToHistory(self, prompt):
        self.history.append((prompt, self.estimateLength(prompt)))
        if self.historyLength() > self.memsize:
            self.summarizeStrategy()
    
    def chat(self, prompt):
        self.addToHistory(self.userID + " " + prompt)
        recommendations = recommend_game(prompt)
        games_list = ", ".join([game[0] for game in recommendations])
    
        chat_prompt = f"""
        L'utilisateur cherche un jeu correspondant à : "{prompt}".
        Voici les jeux recommandés : {games_list}.
        Formule une réponse naturelle et engageante.
        """
    
        fullPrompt = 'Historique de conversation :\n'
        for msg, _ in self.history:
            fullPrompt += msg + "\n"
    
        fullPrompt += "\n" + chat_prompt
    
        resp = self.llm.invoke([HumanMessage(content=fullPrompt)])
        content = resp.content
        if content.startswith(self.llmID + ": "):
            content = content[len(self.llmID + ": "):]
    
        self.addToHistory(self.llmID + " " + content)
        return content


## Initialisation du LLM

In [ ]:
llm = ChatOpenAI(
    openai_api_key="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",
    openai_api_base="https://openrouter.ai/api/v1",
    model_name="mistralai/mistral-small-3.1-24b-instruct:free",
)

In [16]:
CONTEXT_LENGTH = 500  
session = ChatSession(CONTEXT_LENGTH, llm)

In [ ]:
print("\n Bienvenue dans le chatbot de recommandation de jeux vidéo avec mémoire !")
print("\n Pose-moi une question (ex: 'Je veux un RPG avec un bon scénario').")
print("\n Tape 'exit' pour quitter.\n")

while True:
    user_input = input("👤 Vous : ")
    if user_input.lower() == "exit":
        print("À bientôt !")
        break
    
    response = session.chat(user_input)
    print(f"🤖 Bot : {response}\n")



 Bienvenue dans le chatbot de recommandation de jeux vidéo avec mémoire !

 Pose-moi une question (ex: 'Je veux un RPG avec un bon scénario').

 Tape 'exit' pour quitter.



👤 Vous :  I want to have an adventure game with blades and blood


🤖 Bot : It sounds like you're in the mood for an exciting adventure filled with action and intensity! Based on your preferences, here are a few games that might suit you perfectly:

1. **Blood & Blade**: This game is known for its thrilling combat and rich storyline, making it a great choice for fans of adventure and blade-wielding action.

2. **Escape from the Vampire's Tomb**: If you enjoy a mix of mystery and adventure with a touch of the supernatural, this could be a fantastic pick. It's got all the right ingredients for an adrenaline-pumping experience.

3. **Spellcasting 201: The Sorcerer's Appliance**: While this game leans more into magic and spellcasting, it still offers plenty of adventure and excitement, especially if you enjoy a blend of traditional adventuring and magical elements.

Each of these games offers a unique twist on the adventure genre, so you might find one (or more!) that really captures your imagination. Happy adventuring!



👤 Vous :  what about love games
